In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
companies = db.companies.find()

In [3]:
import pandas as pd
df = pd.DataFrame(companies)
df = df[['name', 'number_of_employees', 'offices','funding_rounds', 'founded_year', \
         'deadpooled_year', 'total_money_raised', 'category_code',"tag_list"]]
df.head()

,name,number_of_employees,offices,funding_rounds,founded_year,deadpooled_year,total_money_raised,category_code,tag_list
0,AdventNet,600.0,"[{'description': 'Headquarters', 'address1': '...",[],1996.0,2.0,$0,enterprise,
1,Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 888, 'round_code': 'a', 'source_url': ...",2005.0,1.0,$39.8M,web,"wiki, seattle, elowitz, media-industry, media-..."
2,Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '...",[],2005.0,3.0,$0,software,"zoho, officesuite, spreadsheet, writer, projec..."
3,Digg,60.0,"[{'description': None, 'address1': '135 Missis...","[{'id': 1, 'round_code': 'b', 'source_url': 'h...",2004.0,NaN,$45M,news,"community, social, news, bookmark, digg, techn..."
4,Omnidrive,NaN,"[{'description': '', 'address1': 'Suite 200', ...","[{'id': 225, 'round_code': 'angel', 'source_ur...",2005.0,2008.0,$800k,network_hosting,"storage, sharing, edit, online"


In [4]:
df.columns

Index(['name', 'number_of_employees', 'offices', 'funding_rounds',
       'founded_year', 'deadpooled_year', 'total_money_raised',
       'category_code', 'tag_list'],
      dtype='object')

In [5]:
import requests

currencies = requests.get("https://api.exchangeratesapi.io/latest?base=USD").json()

print(currencies)

{'rates': {'CAD': 1.3055728934, 'HKD': 7.8004458315, 'ISK': 126.348640214, 'PHP': 51.3419527419, 'DKK': 6.6545697726, 'HUF': 289.4872938029, 'CZK': 22.7454302274, 'GBP': 0.7991529202, 'RON': 4.2086491306, 'SEK': 9.4492197949, 'IDR': 14110.0044583148, 'INR': 68.5648684797, 'BRL': 3.8120374498, 'RUB': 63.7561301828, 'HRK': 6.5942933571, 'JPY': 108.6045474811, 'THB': 30.7748551048, 'CHF': 0.992153366, 'EUR': 0.8916629514, 'MYR': 4.1405260811, 'BGN': 1.7439144004, 'TRY': 5.7337494427, 'CNY': 6.8812304949, 'NOK': 8.6348640214, 'NZD': 1.5058403923, 'ZAR': 14.1607668301, 'USD': 1.0, 'MXN': 18.9550601872, 'SGD': 1.359518502, 'AUD': 1.4318323674, 'ILS': 3.5751226037, 'KRW': 1178.8408381632, 'PLN': 3.7922425323}, 'base': 'USD', 'date': '2019-07-08'}


In [6]:
def cleanToFloat(s):
    if "M" in s:
        s=s.replace("M","")
        return float(s)*1000000
    if "B" in s:
        s=s.replace("B","")
        return float(s)*1000000000
    if "k" in s:
        s=s.replace("k","")
        return float(s)*1000
    
def cleanMoney(s):
    if not s == None:
        if s.startswith("$"):
            return cleanToFloat(s.replace("$",""))
        elif s.startswith("€"):
            return int(cleanToFloat(s.replace("€",""))/currencies["rates"]["EUR"])
        elif s.startswith("C$"):
            return int(cleanToFloat(s.replace("C$",""))/currencies["rates"]["CAD"])
        elif s.startswith("£"):
            return int(cleanToFloat(s.replace("£",""))/currencies["rates"]["GBP"])
        elif s.startswith("¥"):
            return int(cleanToFloat(s.replace("¥",""))/currencies["rates"]["JPY"])
        elif s.startswith("kr"):
            return int(cleanToFloat(s.replace("kr",""))/currencies["rates"]["SEK"])
        else:
            return int(s)
    return int(s)

df_aux = df.copy()
df_aux["total_money_raised"] = df_aux["total_money_raised"].apply(cleanMoney)
df_aux["total_money_raised"].value_counts()

1000000.0      93
10000000.0     90
5000000.0      84
2000000.0      79
500000.0       62
3000000.0      57
6000000.0      52
100000.0       51
4000000.0      50
7000000.0      46
1500000.0      44
15000000.0     43
250000.0       43
12000000.0     39
20000000.0     38
8000000.0      34
2500000.0      34
11000000.0     33
30000000.0     31
25000000.0     30
9000000.0      30
40000000.0     27
16000000.0     27
600000.0       26
7500000.0      25
6500000.0      25
17000000.0     24
300000.0       23
50000.0        23
1200000.0      22
               ..
8010000.0       1
38300000.0      1
5920000.0       1
98900000.0      1
513000000.0     1
707000.0        1
38400000.0      1
385000.0        1
41043458.0      1
1610000.0       1
641000.0        1
4180000.0       1
4340000.0       1
24700000.0      1
6480000.0       1
2430000.0       1
3680000.0       1
355000.0        1
66168500.0      1
95000000.0      1
64600000.0      1
38900000.0      1
15641.0         1
1150000.0       1
8270000.0 

In [7]:
df2 = pd.DataFrame(df.offices.tolist()).stack().reset_index(level=1, drop=True).rename('office')
df2

0        {'description': 'Headquarters', 'address1': '4...
1        {'description': '', 'address1': '710 - 2nd Ave...
1        {'description': '', 'address1': '270 Lafayette...
2        {'description': 'Headquarters', 'address1': '4...
3        {'description': None, 'address1': '135 Mississ...
4        {'description': '', 'address1': 'Suite 200', '...
5        {'description': None, 'address1': '959 Skyway ...
6        {'description': None, 'address1': '8536 Nation...
7        {'description': 'Headquarters', 'address1': '9...
8        {'description': None, 'address1': None, 'addre...
9        {'description': '', 'address1': '', 'address2'...
9        {'description': '', 'address1': '', 'address2'...
10       {'description': None, 'address1': '10960 Wilsh...
11       {'description': 'Lala Headquarters', 'address1...
12       {'description': '', 'address1': '407 N Maple D...
13       {'description': '', 'address1': '16935 W. Bern...
14       {'description': 'HQ', 'address1': '539 Bryant .

In [8]:
ndf = df_aux.merge(df2, left_index=True, right_index=True)
display(ndf)

,name,number_of_employees,offices,funding_rounds,founded_year,deadpooled_year,total_money_raised,category_code,tag_list,office
0,AdventNet,600.0,"[{'description': 'Headquarters', 'address1': '...",[],1996.0,2.0,NaN,enterprise,,"{'description': 'Headquarters', 'address1': '4..."
1,Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 888, 'round_code': 'a', 'source_url': ...",2005.0,1.0,3.980000e+07,web,"wiki, seattle, elowitz, media-industry, media-...","{'description': '', 'address1': '710 - 2nd Ave..."
1,Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 888, 'round_code': 'a', 'source_url': ...",2005.0,1.0,3.980000e+07,web,"wiki, seattle, elowitz, media-industry, media-...","{'description': '', 'address1': '270 Lafayette..."
2,Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '...",[],2005.0,3.0,NaN,software,"zoho, officesuite, spreadsheet, writer, projec...","{'description': 'Headquarters', 'address1': '4..."
3,Digg,60.0,"[{'description': None, 'address1': '135 Missis...","[{'id': 1, 'round_code': 'b', 'source_url': 'h...",2004.0,NaN,4.500000e+07,news,"community, social, news, bookmark, digg, techn...","{'description': None, 'address1': '135 Mississ..."
4,Omnidrive,NaN,"[{'description': '', 'address1': 'Suite 200', ...","[{'id': 225, 'round_code': 'angel', 'source_ur...",2005.0,2008.0,8.000000e+05,network_hosting,"storage, sharing, edit, online","{'description': '', 'address1': 'Suite 200', '..."
5,Postini,NaN,"[{'description': None, 'address1': '959 Skyway...",[],1999.0,NaN,NaN,web,,"{'description': None, 'address1': '959 Skyway ..."
6,Flektor,NaN,"[{'description': None, 'address1': '8536 Natio...",[],NaN,NaN,NaN,games_video,"flektor, photo, video","{'description': None, 'address1': '8536 Nation..."
7,Geni,18.0,"[{'description': 'Headquarters', 'address1': '...","[{'id': 6, 'round_code': 'a', 'source_url': ''...",2006.0,NaN,1.650000e+07,web,"geni, geneology, social, family, genealogy","{'description': 'Headquarters', 'address1': '9..."
8,Gizmoz,NaN,"[{'description': None, 'address1': None, 'addr...","[{'id': 9, 'round_code': 'a', 'source_url': 'h...",2003.0,NaN,1.810000e+07,web,"avatar, widget, 3d, facial, taylor","{'description': None, 'address1': None, 'addre..."


In [9]:
ndf = ndf [["name", "number_of_employees", "office", "funding_rounds", "founded_year", "deadpooled_year"\
           ,"total_money_raised", "category_code", "tag_list"]]

ndf.reset_index(inplace=True)

In [10]:
def getCoords(data):
    data = data['office']
    
    principal = None
    if data['latitude'] and data['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data['longitude'], data['latitude']]
        }

    return {
        "lat": data['latitude'],
        "lng": data['longitude'],
        "oficina": principal
    }


coordinates = ndf[["office"]].apply(getCoords, result_type="expand", axis=1)





In [11]:
display (coordinates)

,lat,lng,oficina
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
5,NaN,NaN,None
6,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
7,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
8,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
9,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."


In [12]:
df_clean = pd.concat([ndf,coordinates], axis=1)[["name", "number_of_employees", "founded_year", "deadpooled_year"\
           ,"total_money_raised", "category_code", "tag_list","lat","lng","oficina"]]

In [13]:
df_clean.head()

,name,number_of_employees,founded_year,deadpooled_year,total_money_raised,category_code,tag_list,lat,lng,oficina
0,AdventNet,600.0,1996.0,2.0,NaN,enterprise,,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,47.0,2005.0,1.0,39800000.0,web,"wiki, seattle, elowitz, media-industry, media-...",47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,Wetpaint,47.0,2005.0,1.0,39800000.0,web,"wiki, seattle, elowitz, media-industry, media-...",40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,Zoho,1600.0,2005.0,3.0,NaN,software,"zoho, officesuite, spreadsheet, writer, projec...",37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,Digg,60.0,2004.0,NaN,45000000.0,news,"community, social, news, bookmark, digg, techn...",37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."


In [14]:
print(df_clean.shape)
df_clean.dropna(subset=['lat', 'lng'], inplace=True)
print(df_clean.shape)

(16705, 10)
(10834, 10)


In [21]:
df_clean.to_json('./companies_cleaned.json', orient="records")

In [34]:
# Compañías de game_video
companies_game = db.companies_cleaned.find({"category_code":"games_video"})

# Vemos si son GROW
companies_not_deadpool = db.companies_cleaned.find({"deadpooled_year": None})
companies_deadpool = db.companies_cleaned.find({"deadpooled_year": {"$ne": None}})

# Compañías de design
companies_design = db.companies_cleaned.find({"category_code":"design"})

# Compañías > $1M
companies_one_million = db.companies_cleaned.find({"total_money_raised":{"$gte":1000000}})

# Compañías > $1M
companies_old = db.companies_cleaned.find({"founded_year":{"$gte":2009}})
companies_young = db.companies_cleaned.find({"founded_year":{"$lte":2009}})





In [35]:
df1 = pd.DataFrame(companies_game)
df2 = pd.DataFrame(companies_not_deadpool)
df3 = pd.DataFrame(companies_deadpool)
df4 = pd.DataFrame(companies_design)
df5 = pd.DataFrame(companies_one_million)
df6 = pd.DataFrame(companies_old)
df7 = pd.DataFrame(companies_young)#.to_csv("./companies_young.csv")


In [39]:
df1["type"] = "game"
df2["type"] = "not deadpool"
df3["type"] = "deadpool"
df4["type"] = "design"
df5["type"] = "one million"
df6["type"] = "old"
df7["type"] = "young"


In [40]:
pdList = [df1, df2, df3, df4, df5, df6, df7]  # List of your dataframes
new_df = pd.concat(pdList).to_csv("./companies_with_filters.csv")